# IG Fractal Tree (Theory-Accurate)

Impulse space: U(s) = (ρ(s), κ(s)) ∈ R²
State: X(s) = (x(s), y(s), θ(s)) ∈ R³

Generator vector field:
x'(s) = ρ(s) cos θ(s)
y'(s) = ρ(s) sin θ(s)
θ'(s) = κ(s)

Branching event at s = s_b:
- State inheritance: X_child(0) = X_parent(s_b)
- Impulse inheritance (binary): ρ_child(s) = ρ_parent(s), κ_child(s) = ± κ_parent(s)
- No geometric angle offsets at the branch point.


In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt


In [11]:
A = 0.72     # 0 < A < 1
k0 = 1.2

def rho(s):
    # exponential amplitude scaling (Section 4.2)
    return A**s

def kappa(s):
    # smooth angular impulse (constant is analytic)
    return k0



In [12]:
def impulse(state):
    """
    Impulse space element U(s) = (rho(s), kappa(s))
    """
    rho_val = rho(state.s)
    kappa_val = kappa(state.s)
    return rho_val, kappa_val


In [13]:
from iglab.core.state import State

root = State(
    x=np.array([0.0, 0.0]),
    s=0.0,
    rho=None,          # not used directly anymore
    theta=np.pi / 2   # initial orientation
)


NameError: name 'Integrator' is not defined

In [8]:
def grow_tree(state0, impulse_fn, depth, dt=0.01, steps_per_gen=100):
    segments = []

    def recurse(state, fn, level):
        sys = SinglePathSystem(copy.deepcopy(state), fn, dt)
        sys.run(steps_per_gen)
        traj = np.array(sys.trajectory())
        segments.append(traj)

        if level == 0:
            return

        end_state = sys.state
        for child_state, child_fn in spawn_children(end_state):
            recurse(child_state, child_fn, level-1)

    recurse(state0, impulse_fn, depth)
    return segments


In [9]:
segments = grow_tree(
    state0=root,
    impulse_fn=impulse,
    depth=8,
    dt=0.01,
    steps_per_gen=120
)

plt.figure(figsize=(7,7))
for seg in segments:
    plt.plot(seg[:,0], seg[:,1], lw=1)

plt.axis("equal")
plt.title("IG Fractal Tree (theory-accurate)")
plt.show()


NameError: name 'root' is not defined